In [3]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine
import bcrypt


In [4]:
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_ENTRADA = os.getenv("DB_ENTRADA")
DB_SAI = os.getenv("DB_SAI")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

ENTRADA = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_ENTRADA}"
SAIDA = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_SAI}'

engine_entrada = create_engine(ENTRADA)
engine_saida = create_engine(SAIDA)


query = "SELECT * FROM Analista"
dados_originais_df = pd.read_sql(query, engine_entrada)
df_anonimizado = dados_originais_df.copy()


In [5]:
df_anonimizado["nome_completo"] = list(map(
    lambda i: f"Analista {i+1}", 
    range(len(df_anonimizado))
))




In [6]:
def criptografar(senha):
    
    salt = bcrypt.gensalt()
    hash_senha = bcrypt.hashpw(senha.encode('utf-8'), salt)
    return hash_senha.decode('utf-8')

def verificar_senha(senha_digitada, senha_criptografada):

    return bcrypt.checkpw(senha_digitada.encode('utf-8'), senha_criptografada.encode('utf-8'))


df_anonimizado["senha"] = df_anonimizado["senha"].apply(criptografar)






In [7]:
# toda vez que for rodar esse codigo, TEM QUE RODAR o load_dotenv ANTES!! TODAS AS VEZES!!
def generalizar_data(dt):
    if pd.isna(dt):
        return "Data inválida"
    
    ano = dt.year  
    if ano < 2020:
        return "Antes de 2020"
    elif ano <= 2024:
        return "Entre 2020–2024"
    else:
        return "Após 2024"


df_anonimizado["dt_contratacao"] = df_anonimizado["dt_contratacao"].map(generalizar_data)


In [8]:
def anonimizar_cargo(cargo):
   
    texto = str(cargo).strip().replace(" ", "").replace("-", "")
    return "*" * len(texto)


df_anonimizado["cargo"] = df_anonimizado["cargo"].map(anonimizar_cargo)




In [9]:
def anonimizar_cpf(valor):
    valor_str = str(valor)
    valor_anonimizado = ''.join('*' if caractere.isdigit() else caractere for caractere in valor_str)
    return valor_anonimizado

df_anonimizado["cpf"] = df_anonimizado["cpf"].map(anonimizar_cpf)



In [10]:

def pseudonimizar_email(email):
    email = str(email)
    parte_local, dominio = email.split("@")
  
    if len(parte_local) > 3:
        parte_local_pseudo = parte_local[:0] + "****"
    else:
        parte_local_pseudo = parte_local + "****"
    return parte_local_pseudo + "@" + dominio

df_anonimizado["email"] = df_anonimizado["email"].apply(pseudonimizar_email)



In [11]:
def anonimizar_telefone(valor):
    valor_str = str(valor)
    valor_anonimizado = ''.join('*' if caractere.isdigit() else caractere for caractere in valor_str)
    return valor_anonimizado

df_anonimizado["telefone"] = df_anonimizado["telefone"].map(anonimizar_telefone)


In [12]:
def anonimizar_id_unidade(valor):
    valor_str = str(valor)
    valor_anonimizado = ''.join('*' if caractere.isdigit() else caractere for caractere in valor_str)
    return valor_anonimizado

df_anonimizado["id_unidade"] = df_anonimizado["id_unidade"].map(anonimizar_id_unidade)




In [13]:
display(df_anonimizado)

,id,id_unidade,cpf,dt_contratacao,nome_completo,email,senha,cargo,telefone
0,1,*,***********,Entre 2020–2024,Analista 1,****@eitruck.com,$2b$12$.0oHMKIUBylGINea88X4ueEAc7ulKQ5fat6TSqO...,*************,***********
1,2,*,***********,Entre 2020–2024,Analista 2,****@eitruck.com,$2b$12$.gG4Yzp4EDHy3SHPTsm/qO/vZqK3zqzcp13K63n...,****************,***********
2,3,*,***********,Entre 2020–2024,Analista 3,****@eitruck.com,$2b$12$a/IOeW9ftZ2jgmwIOVU3geyd/fijUr7jwvcOFBO...,****************,***********
3,4,*,***********,Entre 2020–2024,Analista 4,****@eitruck.com,$2b$12$gB.0Pml/tcHLQ.NaKHhxTe91V.oOoncFl/OLy4w...,****************,***********
4,5,*,***********,Entre 2020–2024,Analista 5,****@eitruck.com,$2b$12$nGOybeIuuT/bLERqNYrERu1ZqbG9beDJHh4vTtC...,*************,***********


In [ ]:
df_anonimizado.to_sql(name='analista', con=engine_saida, if_exists='replace', index=False)